In [24]:
from __future__ import print_function

## Naming conventions

The python community has some naming convections, defined in PEP-8:

https://www.python.org/dev/peps/pep-0008/

The widely adopted ones are:

* class names start with an uppercase, and use "camelcase" for multiword names, e.g. `ShoppingCart`

* varible names (including objects which are instances of a class) are lowercase and use underscores to separate words, e.g., `shopping_cart`

* module names should be lowercase with underscores



# Practicing Classes

## Exercise 1 (shopping cart)

Let's write a simple shopping cart class -- this will hold items that you intend to purchase as well as the amount, etc.  And allow you to add / remove items, get a subtotal, etc.

We'll use two classes: `Item` will be a single item and `ShoppingCart` will be the collection of items you wish to purchase.

First, our store needs an inventory -- here's what we have for sale:

In [112]:
INVENTORY_TEXT = """
apple, 0.60
banana, 0.20
grapefruit, 0.75
grapes, 1.99
kiwi, 0.50
lemon, 0.20
lime, 0.25
mango, 1.50
papaya, 2.95
pineapple, 3.50
blueberries, 1.99
blackberries, 2.50
peach, 0.50
plum, 0.33
clementine, 0.25
cantaloupe, 3.25
pear, 1.25
quince, 0.45
orange, 0.60
"""

# this will be a global -- convention is all caps
INVENTORY = {}
for line in INVENTORY_TEXT.splitlines():
    if line.strip() == "":
        continue
    item, price = line.split(",")
    INVENTORY[item] = float(price)


In [113]:
INVENTORY

{'apple': 0.6,
 'banana': 0.2,
 'blackberries': 2.5,
 'blueberries': 1.99,
 'cantaloupe': 3.25,
 'clementine': 0.25,
 'grapefruit': 0.75,
 'grapes': 1.99,
 'kiwi': 0.5,
 'lemon': 0.2,
 'lime': 0.25,
 'mango': 1.5,
 'orange': 0.6,
 'papaya': 2.95,
 'peach': 0.5,
 'pear': 1.25,
 'pineapple': 3.5,
 'plum': 0.33,
 'quince': 0.45}

### `Item` 

Here's the start of an item class -- we want it to hold the name and quantity.  

You should have the following features:

* the name should be something in our inventory

* Our shopping cart will include a list of all the items we want to buy, so we want to be able to check for duplicates.  Implement the equal test, `==`, using `__eq__`

* we'll want to consolidate dupes, so implement the `+` operator, using `__add__` so we can add items together in our shopping cart.  Note, add should raise a ValueError if you try to add two `Items` that don't have the same name.

Here's a start:

In [114]:
class Item(object):
    """ an item to buy """
    
    def __init__(self, name, quantity=1):
        """keep track of an item that is in our inventory"""
        if name not in INVENTORY:
            raise ValueError("invalid item name")
        self.name = name
        self.quantity = quantity
        
    def __repr__(self):
        return "{}: {}".format(self.name, self.quantity)
        
    def __eq__(self, other):
        """check if the items have the same name"""
        return self.name == other.name
    
    def __add__(self, other):
        """add two items together if they are the same type"""
        if self.name == other.name:
            return Item(self.name, self.quantity + other.quantity)
        else:
            raise ValueError("names don't match")

Here are some tests your code should pass:

In [115]:
a = Item("apple", 10)
b = Item("banana", 20)

In [116]:
a.name

'apple'

In [117]:
a.quantity

10

In [118]:
c = Item("apple", 20)

In [119]:
# won't work
a + b

ValueError: names don't match

In [120]:
# will work
a += c
print(a)

apple: 30


In [121]:
d = Item("dog")

ValueError: invalid item name

In [122]:
# should be False
a == b

False

In [123]:
# should be True -- they have the same name
a == c

True

How do they behave in a list?

In [124]:
items = []
items.append(a)
items.append(b)
items

[apple: 30, banana: 20]

In [125]:
# should be True -- they have the same name
c in items

True

### `ShoppingCart`

Now we want to create a shopping cart.  The main thing it will do is hold a list of items.

__Note__: I used the outline of the `ShoppingCart` class you provided, and just filled in the blanks. Everything above this was already in the notebook.

In [126]:
class ShoppingCart(object):
    
    def __init__(self):
        # the list of items we control
        self.items = []
        
    def subtotal(self):
        """ return a subtotal of our items """
        price = 0.0
        for item in self.items:
            item_price = INVENTORY[item.name]    # use the item's name to get its price       
            price += item_price * item.quantity  # multiply the price by the quantity and add to total
        return price

    def add(self, name, quantity):
        """ add an item to our cart -- if an item of the same name already
        exists, then increment the quantity.  Otherwise, add a new item
        to the cart with the desired quantity."""
        new_item = Item(name, quantity=quantity) # create new Item object
        if new_item in self.items:
            idx = self.items.index(new_item)     # find position of the item in the list
            self.items[idx] += new_item          # add the new quantity to the item
        else:
            self.items.append(new_item)          # add the item to the shopping cart
        
    def remove(self, name):
        """ remove all of item name from the cart """
        item_names = [item.name for item in self.items]
        idx = item_names.index(name)      # get index of item in list
        self.items.pop(idx)                      # remove it
        
    def report(self):
        """ print a summary of the cart """
        for item in self.items:
            print("{} : {}".format(item.name, item.quantity))

Here are some tests

In [127]:
sc = ShoppingCart()
sc.add("orange", 19)

In [128]:
sc.add("apple", 2)

In [129]:
sc.report()

orange : 19
apple : 2


In [130]:
sc.add("apple", 9)

In [131]:
# apple should only be listed once in the report, with a quantity of 11
sc.report()

orange : 19
apple : 11


In [132]:
sc.subtotal()

18.0

In [133]:
sc.remove("apple")

In [134]:
# apple should no longer be listed
sc.report()

orange : 19


## Exercise 2: Poker Odds

Use the deck of cards class from the notebook we worked through outside of class to write a _Monte Carlo_ code that plays a lot of hands of straight poker (like 100,000).  Count how many of these hands has a particular poker hand (like 3-of-a-kind).  The ratio of # of hands with 3-of-a-kind to total hands is an approximation to the odds of getting a 3-of-a-kind in poker.

You'll want to copy-paste those classes into a `.py` file to allow you to import and reuse them here

In [5]:
# copied and pasted this cell from the other notebook for now
import random


class Card(object):
    
    def __init__(self, suit=1, rank=2):
        if suit < 1 or suit > 4:
            print("invalid suit, setting to 1")
            suit = 1
        if rank < 2 or rank > 14:
            print("invalid rank, setting to 2")
            rank = 2
        
        self.suit = suit
        self.rank = rank
        

    def value(self):
        """ we want things order primarily by rank then suit """
        return self.suit + (self.rank-1)*14
    
    # we include this to allow for comparisons with < and > between cards 
    def __lt__(self, other):
        return self.value() < other.value()

    def __unicode__(self):
        suits = [u"\u2660".encode('utf-8'),  # spade
                 u"\u2665".encode('utf-8'),  # heart
                 u"\u2666".encode('utf-8'),  # diamond
                 u"\u2663".encode('utf-8')]  # club
        
        r = str(self.rank)
        if self.rank == 11:
            r = "J"
        elif self.rank == 12:
            r = "Q"
        elif self.rank == 13:
            r = "K"
        elif self.rank == 14:
            r = "A"
                
        return r +':'+suits[self.suit-1]
    
    def __str__(self):
        return self.__unicode__()  #.encode('utf-8')
        
        


class Deck(object):
    """ the deck is a collection of cards """

    def __init__(self):

        self.nsuits = 4
        self.nranks = 13
        self.minrank = 2
        self.maxrank = self.minrank + self.nranks - 1

        self.cards = []

        for rank in range(self.minrank,self.maxrank+1):
            for suit in range(1, self.nsuits+1):
                self.cards.append(Card(rank=rank, suit=suit))

    def shuffle(self):
        random.shuffle(self.cards)

    def get_cards(self, num=1):
        hand = []
        for n in range(num):
            hand.append(self.cards.pop())

        return hand
    
    def __str__(self):
        string = ""
        for c in self.cards:
            string += str(c) + " "
        return string

I've added lots of comments, but the basic idea is this:

We are counting any deck of cards that has exactly 3 cards of the same rank. We will investigate 100,000 different hands (5 random cards, chosen randomly from a full deck of 52 cards). For each iteration,

- Generate a new deck of cards, shuffle it, and draw a hand.
- We will have, at most, 5 unique ranks. Create a dictionary, whose keys will be the set of unique ranks in the deck, and whose values will be the number of cards with that rank.
    - For example, if the deck had ranks `5, 9, 8, 4, 9`, the dictionary would be `{5: 1, 9: 2, 8: 1, 4: 1}`.
- For each card in the deck:
    - Get its rank.
    - Check if that rank is already a key in the dictionary.
        - If it's not, add that rank as a new key, and initialize its value to 1.
        - If it is, add 1 to the value of that key.
- Get a list of the values in the dictionary. If one of the values is `3`, then there are 3 cards with the same rank. Add one to the counter that's keeping track of how many hands have 3-of-a-kind.

In [18]:
def n_of_a_kind(nkind=3, ncards=5, N=100000):
    """Determine how often a random hand of cards will contain nkind cards of the same rank.
    Parameters
    ----------
    nkind : int
        The necessary number of cards with the same rank.
    ncards : int
        The number of cards to draw for each hand.
    N : int
        The number of hands to draw.
    Returns
    -------
    ratio : float
        The ratio of the number of hands with nkind cards of the same rank to the total number N."""
    count = 0 # count number of hands w/ n of a kind

    # generate N decks of cards
    for i in range(N):
        # generate the deck, shuffle the cards, and get a new hand:
        deck = Deck()
        deck.shuffle() 
        hand = deck.get_cards(ncards)

        # create an empty dictionary:
        # the keys will be the unique rank(s) of cards in the deck
        # the values will be the number of cards in the hand with that rank
        ranks = {}
        for card in hand:
            if ranks.has_key(card.rank):
                # if the rank is already in the dict, add one to the count for that rank
                ranks[card.rank] += 1
            else:
                # otherwise, this is first card with that rank
                ranks[card.rank] = 1 # initialze value to 1 card with that rank

        # we don't actually care about the specific rank, only the number of cards with the same rank.
        # so, ranks.values() will give a list of the values in the dict (no need for the keys anymore).
        # if this list contains an element that is the same as nkind, then there are nkind cards with same rank.
        # example: here nkind=3, so if any of the values in the dict are 3, we have 3 cards of the same kind.
        if nkind in ranks.values():
            count += 1

    ratio = float(count)/float(N)
    print("Out of {} hands, {} had 3-of-a-kind, or {}%".format(N, count, ratio*100.))
    return ratio

In [19]:
N = 100000 # number of hands
ncards = 5 # number of cards per hand
nkind = 3 # looking for 3 of a kind

# repeat a few times and look at distribution of results
nruns = 10
ratios = []
for i in range(nruns):
    ratios.append(n_of_a_kind(nkind=nkind, ncards=ncards, N=N))

Out of 100000 hands, 2220 had 3-of-a-kind, or 2.22%
Out of 100000 hands, 2256 had 3-of-a-kind, or 2.256%
Out of 100000 hands, 2196 had 3-of-a-kind, or 2.196%
Out of 100000 hands, 2249 had 3-of-a-kind, or 2.249%
Out of 100000 hands, 2359 had 3-of-a-kind, or 2.359%
Out of 100000 hands, 2259 had 3-of-a-kind, or 2.259%
Out of 100000 hands, 2345 had 3-of-a-kind, or 2.345%
Out of 100000 hands, 2248 had 3-of-a-kind, or 2.248%
Out of 100000 hands, 2183 had 3-of-a-kind, or 2.183%
Out of 100000 hands, 2276 had 3-of-a-kind, or 2.276%


In [23]:
avg_ratio = sum(ratios) / float(nruns)
print("Average chance of getting 3-of-a-kind is {:1.2f}%.".format(avg_ratio*100.))

Average chance of getting 3-of-a-kind is 2.26%.


## Exercise 3: Tic-Tac-Toe

Revisit the tic-tac-toe game you developed in the functions exercises but now write it as a class with methods to do each of the main steps.  

In [1]:
class TicTacToe(object):
    """Let's play a game of tic-tac-toe!"""
    
    def __init__(self, player1_name="Player 1", player2_name="Player 2"):
        """Initialize a new game of tic-tac-toe.
        Parameters
        ----------
        player1_name : string, optional
            The name of the first player.
        player2_name : string, optional
            The name of the second player."""
        
        # optional names of players
        self.name1 = player1_name
        self.name2 = player2_name
        
        # give player 1 "x"'s and player 2 "o"'s
        self.piece1 = 'x'
        self.piece2 = 'o'
        
        # create the empty board
        self.board = """
         {s1:^3} | {s2:^3} | {s3:^3}
        -----+-----+-----
         {s4:^3} | {s5:^3} | {s6:^3}
        -----+-----+-----      123
         {s7:^3} | {s8:^3} | {s9:^3}       456
                               789  
        """
        
        self.play = {} # dictionary holding the status of each square: "x", "o", or empty ("")
        self.initialize_board() # initialize the game
        
        # keep track of the game progress
        self.have_winner = False
        self.have_tie = False
        self.winner = None
        
    
    def initialize_board(self):
        """Fill in the tic-tac-toe board with empty strings to start a game."""
        for n in range(9):
            self.play["s{}".format(n+1)] = ""  # "s1" for square 1, etc.
            
            
    def show_board(self):
        """ Display the current state of the playing board."""
        print(self.board.format(**self.play)) # same as board.format(s1=play['s1'], s2=play['s2], ...)
        
        
    def get_move(self, n, xo):
        """ Ask the current player to make a move and make sure the square was not 
            already played.  
            Parameters:
            -----------
            xo: is a string of the character (x or o) we will place in
            the desired square """
        valid_move = False
        print("Asking for input...")
        while not valid_move:
            idx = input("player {}, enter your move (1-9)".format(n)) # ask which square they want to play
            print("You've entered: {}".format(idx))
            if self.play["s{}".format(idx)] == "":
                valid_move = True # if the square is empty, they're free to play it
            else:
                print("invalid: {}".format(self.play["s{}".format(idx)]))

        self.play["s{}".format(idx)] = xo # update entry in the square
    
    
    
    def check_win(self):
        """Recevies the game board and checks if someone has already won the game.
        Returns:
        -------
        win:  Logical
             true is someone has won"""
        win = False
        if self.play['s1']==self.play['s2']==self.play['s3']==('x'or'o'):
            win = True
        elif self.play['s4']==self.play['s5']==self.play['s6']==('x'or'o'):
            win = True
        elif self.play['s7']==self.play['s8']==self.play['s9']==('x'or'o'):
            win = True
        elif self.play['s1']==self.play['s4']==self.play['s7']==('x'or'o'):
            win = True    
        elif self.play['s5']==self.play['s2']==self.play['s8']==('x'or'o'):
            win = True
        elif self.play['s9']==self.play['s6']==self.play['s3']==('x'or'o'):
            win = True    
        elif self.play['s5']==self.play['s7']==self.play['s3']==('x'or'o'):
            win = True
        elif self.play['s1']==self.play['s5']==self.play['s9']==('x'or'o'):
            win = True     
        return win    
    
    
    def check_for_tie(self):
        """Check if there is a tie in the tic-tac-toe game.
        Note: this differs from check_win(), because game may be over without a winner.
        Returns:
        -------
        game_over: Logical
            True if game over (all squares occupied).
        """
        items = [self.play[key] for key in self.play.keys()]
        # if each box filled, we have a tie.
        # begin by assuming this is the case, and if a box is empty, keep playing
        game_over = True
        for item in items:
            if item == '':
                game_over = False
        return game_over
    
    
    def play_game(self):
        """Play a game of tic-tac-toe!"""

        # if the game has been played previously (the board has been filled), don't play another game
        if (not self.have_winner) or (not self.have_tie):
            welcome_message = "Beginning game. The board is currently empty. {} has '{}'s, {} has '{}'s"
            print(welcome_message.format(self.name1, self.piece1, self.name2, self.piece2))
            self.show_board()
        
        # play until someone wins:
        while not (self.have_winner or self.have_tie):
            # ask player 1 to make move
            self.get_move(self.name1, self.piece1)
            print("The board is now:")
            self.show_board()
            # check if game over
            self.have_winner = self.check_win()
            self.have_tie = self.check_for_tie()
            if self.have_winner:
                self.winner = self.name1
                break
            elif self.have_tie:
                break

            # ask player 2
            self.get_move(self.name2, self.piece2)
            print("The board is now:")
            self.show_board()
            # check if game over
            self.have_winner = self.check_win()
            self.have_tie = self.check_for_tie()
            if self.have_winner:
                self.winner = self.name2
            # don't need to check for a tie again, because looping back to top

        if self.winner is None:
            print("There is a tie!")
        else:
            print("We have a winner! {}, you've won the game.".format(self.winner))
        print("Here is the final board:")
        self.show_board()


In [2]:
# testing
game = TicTacToe() # initialize a game
print(game.play)  # should be empty
print(game.board) # should be empty
game.show_board() # should be filled in (with empty strings)


{'s9': '', 's8': '', 's3': '', 's2': '', 's1': '', 's7': '', 's6': '', 's5': '', 's4': ''}

         {s1:^3} | {s2:^3} | {s3:^3}
        -----+-----+-----
         {s4:^3} | {s5:^3} | {s6:^3}
        -----+-----+-----      123
         {s7:^3} | {s8:^3} | {s9:^3}       456
                               789  
        

             |     |    
        -----+-----+-----
             |     |    
        -----+-----+-----      123
             |     |           456
                               789  
        


In [3]:
game.play_game()

Beginning game. The board is currently empty. Player 1 has 'x's, Player 2 has 'o's

             |     |    
        -----+-----+-----
             |     |    
        -----+-----+-----      123
             |     |           456
                               789  
        
Asking for input...
player Player 1, enter your move (1-9)1
You've entered: 1
The board is now:

          x  |     |    
        -----+-----+-----
             |     |    
        -----+-----+-----      123
             |     |           456
                               789  
        
Asking for input...
player Player 2, enter your move (1-9)2
You've entered: 2
The board is now:

          x  |  o  |    
        -----+-----+-----
             |     |    
        -----+-----+-----      123
             |     |           456
                               789  
        
Asking for input...
player Player 1, enter your move (1-9)3
You've entered: 3
The board is now:

          x  |  o  |  x 
        -----+-----+----

In [4]:
game.play_game()

We have a winner! Player 1, you've won the game. Here is the final board:

          x  |  o  |  x 
        -----+-----+-----
          o  |  x  |  o 
        -----+-----+-----      123
          x  |     |           456
                               789  
        
